<a href="https://colab.research.google.com/github/ayush-gupta0/Enterprise-RAG-MLOps-2026/blob/main/Enterprise_RAG_MLOps_2026.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install modular LangChain packages and MLOps tools
!pip install -q -U langchain-groq langchain-community langchain-text-splitters chromadb sentence-transformers pypdf mlflow pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.6/329.6 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 118.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.8 MB/s eta 0:

In [2]:
import os
import getpass

# 1. Securely set Groq Key
os.environ["GROQ_API_KEY"] = getpass.getpass("Enter Groq API Key (starts with gsk_): ").strip()

# 2. Securely set ngrok Token
NGROK_TOKEN = getpass.getpass("Enter ngrok Authtoken: ").strip()

Enter Groq API Key (starts with gsk_): ··········
Enter ngrok Authtoken: ··········


In [6]:
import subprocess
from pyngrok import ngrok, conf
import getpass

# 1. Kill old tunnels and set token
ngrok.kill()
conf.get_default().auth_token = getpass.getpass("Enter ngrok Authtoken: ")

# 2. Start MLflow with a stable backend
# Using 0.0.0.0 allows it to listen to external requests from the tunnel
subprocess.Popen(["mlflow", "ui", "--port", "5000", "--host", "0.0.0.0", "--backend-store-uri", "sqlite:///mlflow.db"])

# 3. Create the tunnel with the HOST HEADER REWRITE (The fix)
# This trick makes MLflow believe the request is local
public_url = ngrok.connect(5000, pyngrok_config=conf.get_default(), host_header="localhost:5000").public_url

print(f"🚀 Fixed MLflow Dashboard: {public_url}")

Enter ngrok Authtoken: ··········
🚀 Fixed MLflow Dashboard: https://linguistical-jellylike-michaele.ngrok-free.dev


In [5]:
import mlflow
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# --- Configuration ---
FILE_PATH = "data.pdf" # Ensure you upload 'data.pdf' to Colab's folder icon
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("Enterprise_RAG_v2026")

# --- 1. Load & Split ---
loader = PyPDFLoader("/content/data.pdf")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(docs)

# --- 2. Vector Indexing ---
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector_db = Chroma.from_documents(documents=chunks, embedding=embeddings)
retriever = vector_db.as_retriever()

# --- 3. RAG Logic ---
llm = ChatGroq(model_name="llama-3.1-8b-instant", temperature=0)
prompt = ChatPromptTemplate.from_template("Context: {context}\n\nQuestion: {question}")

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt | llm | StrOutputParser()
)

# --- 4. Execution & MLOps Tracking ---
with mlflow.start_run(run_name="Llama_3.1_Production_Run"):
    question = "Summarize the main objective of this document."
    response = rag_chain.invoke(question)

    # Log to MLflow
    mlflow.log_params({"model": "llama-3.1-8b", "chunk_size": 1000})
    mlflow.log_metric("context_chunks", len(chunks))
    mlflow.log_text(response, "ai_final_answer.txt")

    print(f"\n✅ AI Response:\n{response}")
    print(f"\n📊 Check your dashboard at the ngrok link above!")

/tmp/ipython-input-2031664564.py:23: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


✅ AI Response:
Based on the provided document metadata and content, it appears to be a thesis or research document. The main objective of this document can be inferred from the content of the first page (page 68 or 69, depending on the label).

The first page content mentions that the framework has been "carefully crafted, diligently implemented, and rigorously tested" to offer significant improvements over conventional approaches. It also mentions a sound foundation for future research studies and practical applications in the discipline.

However, to get a clearer understanding of the main objective, we need to look at the abstract or introduction section of the document. Unfortunately, the provided content does not include the abstract or introduction section.

But, based on the table of contents (page 5 or 6), we can see that the document is structured as follows:

- Chapter 1: Introduction
- 1.1. Background and Motivation
- 1.1.1. Analysis of the Research Problem and the Signific